In [1]:
import pandas as pd

In [2]:
pwd

'c:\\Users\\ekino\\OneDrive - UW\\TransitRecovery'

In [4]:
# Read the data xlsx
df = pd.read_excel('data/stop_level/stop2020_1to6.xlsx')

In [5]:
# Read census data
df_census = pd.read_excel('data/stop_level/stops_census_info.xlsx')

In [9]:
# Merge based on 'STOP_ID'
df_merged = pd.merge(df, df_census, on='STOP_ID')

In [10]:
df_merged

,Unnamed: 0,OPD_DATE,STOP_ID,TOTAL_BOARDING,TOTAL_ALIGHTING,datenew,weekday,month,STOP_STATU,FARE_ZONE,...,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
0,1,2020-01-01,258,1,0,2020-01-01,Wednesday,1,ACT,21,...,8002,53033008002,80.02,Census Tract 80.02,G5020,S,234314,0,47.612090,-122.345362
1,1,2020-01-01,258,1,0,2020-01-01,Wednesday,1,ACT,21,...,8002,53033008002,80.02,Census Tract 80.02,G5020,S,234314,0,47.612090,-122.345362
2,1,2020-01-01,258,1,0,2020-01-01,Wednesday,1,ACT,21,...,8002,53033008002,80.02,Census Tract 80.02,G5020,S,234314,0,47.612090,-122.345362
3,5122,2020-01-02,258,17,2,2020-01-02,Thursday,1,ACT,21,...,8002,53033008002,80.02,Census Tract 80.02,G5020,S,234314,0,47.612090,-122.345362
4,5122,2020-01-02,258,17,2,2020-01-02,Thursday,1,ACT,21,...,8002,53033008002,80.02,Census Tract 80.02,G5020,S,234314,0,47.612090,-122.345362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145642,1038667,2020-06-29,15146,5,7,2020-06-29,Monday,6,INA,1,...,9900,53033009900,99.00,Census Tract 99,G5020,S,6565170,2370735,47.579454,-122.355033
3145643,1038667,2020-06-29,15146,5,7,2020-06-29,Monday,6,INA,1,...,9900,53033009900,99.00,Census Tract 99,G5020,S,6565170,2370735,47.579454,-122.355033
3145644,1044455,2020-06-30,15146,7,3,2020-06-30,Tuesday,6,INA,1,...,9900,53033009900,99.00,Census Tract 99,G5020,S,6565170,2370735,47.579454,-122.355033
3145645,1044455,2020-06-30,15146,7,3,2020-06-30,Tuesday,6,INA,1,...,9900,53033009900,99.00,Census Tract 99,G5020,S,6565170,2370735,47.579454,-122.355033
